In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Find the $\lambda$, $B^2$, and $\alpha$ eigenvalues for infinite medium cross sections

In [2]:
m = np.array([
    [1, 0.0055, 0.072, 2, 2.2e9, 0.07, 0, 0],
    [0, 0.068, 0.20, 0.7, 3.2e8, 0, 0.13, 0],
    [0, 2.48, 2.02, 0.2, 2.7e7, 0, 0, 0.58],
    [0, 2.00, 1.0, 0.3, 2.5e5, 0, 0, 0]
])
cols = ['chi', 'nu-sigma_f', 'sigma-a', 'D', 'v', 's->1', 's->2', 's->3']
df = pd.DataFrame(m, columns=cols)

In [3]:
df

,chi,nu-sigma_f,sigma-a,D,v,s->1,s->2,s->3
0,1.0,0.0055,0.072,2.0,2.200000e+09,0.07,0.00,0.00
1,0.0,0.0680,0.200,0.7,3.200000e+08,0.00,0.13,0.00
2,0.0,2.4800,2.020,0.2,2.700000e+07,0.00,0.00,0.58
3,0.0,2.0000,1.000,0.3,2.500000e+05,0.00,0.00,0.00


# Base Equation

$\sigma_{ag} \phi_g = \chi_g + \sigma_s^{(g-1) \rightarrow g}$

$\lambda = k_{eff} = \sum_{g^\prime = 0}^{3} \nu \sigma_{g^\prime} \phi_{g^\prime}$

# Solve for $\lambda$

In [4]:
# Initial guess for flux (for each group)
flux = np.ones(4)

In [5]:
abs_matrix = np.diag(df['sigma-a'])
pp.pprint(abs_matrix)

array([[ 0.072,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.2  ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  2.02 ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  1.   ]])


In [6]:
production_matrix = np.array([
    df['nu-sigma_f'],
    df['s->1'],
    df['s->2'],
    df['s->3']
])

In [7]:
pp.pprint(production_matrix)

array([[ 0.0055,  0.068 ,  2.48  ,  2.    ],
       [ 0.07  ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.13  ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.58  ,  0.    ]])


In [8]:
pp.pprint(la.eig(la.inv(production_matrix) @ abs_matrix))

(   array([-2.06044964+0.j        , -0.46841098+1.15623055j,
       -0.46841098-1.15623055j,  0.85934057+0.j        ]),
    array([[-0.80522899+0.j        ,  0.91573087+0.j        ,
         0.91573087-0.j        , -0.95747168+0.j        ],
       [ 0.58069683+0.j        , -0.15012844+0.3705786j ,
        -0.15012844-0.3705786j , -0.28797799+0.j        ],
       [-0.07700225+0.j        , -0.02304942-0.02234238j,
        -0.02304942+0.02234238j, -0.01592636+0.j        ],
       [ 0.09202238+0.j        ,  0.02124513-0.00938732j,
         0.02124513+0.00938732j, -0.00793798+0.j        ]]))


# $\alpha$

$\alpha$ is inserted to "bump up" the absorption term such that $(\frac{\alpha}{v}+\sigma_a)\phi = (\chi \nu \sigma_f + \sigma_s^{g^\prime \rightarrow g}) \phi$ is solved. This means that the eigenvalues of $\chi \nu \sigma_f + \sigma_s^{g^\prime \rightarrow g} - \sigma_a$ are the $\frac{\alpha}{v}$ term and the eigenvectors are the flux.

In [9]:
alpha_matrix = production_matrix - abs_matrix

In [10]:
alpha_matrix

array([[-0.0665,  0.068 ,  2.48  ,  2.    ],
       [ 0.07  , -0.2   ,  0.    ,  0.    ],
       [ 0.    ,  0.13  , -2.02  ,  0.    ],
       [ 0.    ,  0.    ,  0.58  , -1.    ]])

In [11]:
la.eig(alpha_matrix)

(array([-2.01655591,  0.02573986, -0.31036343, -0.98532052]),
 array([[ 0.51259883, -0.95490944,  0.84326884, -0.91140237],
        [-0.01975272, -0.29610925, -0.5348585 ,  0.08123838],
        [-0.74558152, -0.01881676, -0.0406704 ,  0.01020702],
        [ 0.42539449, -0.01063985, -0.03420473,  0.4032888 ]]))

We choose the largest eigenvalue, 0.0257, except that has negative flux values what does that even mean?

In [12]:
v_mat = np.diag(df['v']**-1)

In [13]:
v_mat

array([[  4.54545455e-10,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   3.12500000e-09,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   3.70370370e-08,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.00000000e-06]])

In [14]:
la.eig(la.inv(v_mat) @ alpha_matrix)

(array([ -1.35072321e+08+25943628.35003445j,
         -1.35072321e+08-25943628.35003445j,
          5.74828594e+06       +0.j        ,  -6.93643677e+05       +0.j        ]),
 array([[  9.58792370e-01 +0.00000000e+00j,
           9.58792370e-01 -0.00000000e+00j,
          -9.51953535e-01 +0.00000000e+00j,
          -9.42479033e-01 +0.00000000e+00j],
        [ -2.66653420e-01 -9.73368693e-02j,
          -2.66653420e-01 +9.73368693e-02j,
          -3.05724491e-01 +0.00000000e+00j,
          -3.33482000e-01 +0.00000000e+00j],
        [  9.29113054e-03 +7.23557996e-03j,
           9.29113054e-03 -7.23557996e-03j,
          -1.77993609e-02 +0.00000000e+00j,
          -2.17381806e-02 +0.00000000e+00j],
        [ -8.19174797e-06 -9.35811480e-06j,
          -8.19174797e-06 +9.35811480e-06j,
          -4.30274142e-04 +0.00000000e+00j,
           7.10488249e-03 +0.00000000e+00j]]))